In [28]:
library(dplyr)

In [29]:
df <- read.csv("BuySell.csv", header = TRUE, sep = ",")
nrow(df)
head(df,3)
tail(df,3)

[1] 41

,date,type,co,quan,pps,basis,tally_q,tally_b,Sale
,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,44926,Owned,AMC,1050,4.070,4273.5,1050,4273.5,0
2,44946,Buy,AMC,1000,5.625,5625.0,2050,5625.0,0
3,44949,Buy,AMC,1000,5.788,5788.0,3050,5788.0,0


,date,type,co,quan,pps,basis,tally_q,tally_b,Sale
,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
39,45063,Buy,AMC,1000,5.01,5009.0,2100,5009.0,0
40,45076,Sell,AMC,-1000,4.58,-4582.8,1100,-4582.8,-1000
41,45079,Sell,AMC,-1100,4.54,-4994.0,0,-4994.0,-1100


In [30]:
df$quan <- as.integer(df$quan)
df$tally_q <- as.integer(df$tally_q)

head(df,3)

,date,type,co,quan,pps,basis,tally_q,tally_b,Sale
,<int>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<dbl>,<int>
1,44926,Owned,AMC,1050,4.070,4273.5,1050,4273.5,0
2,44946,Buy,AMC,1000,5.625,5625.0,2050,5625.0,0
3,44949,Buy,AMC,1000,5.788,5788.0,3050,5788.0,0


In [31]:
# Convert Excel dates when in the 1900 date format

# df$date <- as.Date(df$date, origin = "1899-12-30")
# head(df, 3)

In [32]:
# First allocate, then play with cost basis methods

adj_tally_q <- function(tally_q, quan_sold) {
    # cat("tally_q in the remote fct", tally_q, "\n")
    tally_q <- tally_q + quan_sold
    cat("ADJUSTED tally_q - reduced by proceeds in remote fct", tally_q, "\n")
    return(tally_q)        
    }

## <font color = yellow> Keeper! These values agree with .csv/spreadsheet
    
##### Getting the spreadsheet right w Buy/Sells remains the workout

In [33]:
# Function that reads BuySell.csv and will perform actions which agree with the values of (tally_q_q) on Sell dates

make_buysell_csv <- function() {    

    # remote fct to test passing value from fct to remote to this remote
     adj_basis <- function(tally_q, quan_sold) {
        print(paste0("In adj_basis fct. tally_q is: ", tally_q, " Sell is: ", quan_sold))
        n <- 2
        tally_q <<- tally_q + n
        return(tally_q)
        }   

    # remote fct to adjust the running stock tally_q (combine sum of Buys with the next Sell), return adjusted tally_q
    adj_tally_q <- function(tally_q, quan_sold) {
        cat("tally_q in the remote fct", tally_q, "proceeds is: ", quan_sold, "\n")
        tally_q <- tally_q + quan_sold
        cat("ADJUSTED tally_q - reduced by proceeds in remote fct", tally_q, "\n")
        # Fct call
        # adjusted <- adj_basis(tally_q, proceeds)
        # print(paste0("adjusted from remote fct is: ", adjusted))
        return(tally_q)        
        }       

    # Main body     

    # Create counter vars
    ct <- 0
    tally_q <<- 0
    row_ct <<- 0
    quan_sold <- 0
    
    cat("The df has the following nrows: ", nrow(df), "\n \n")

    # loop through the df
    for(i in 1:nrow(df))  { 

         ct <- ct + 1
        # Sum the buy values from each row
        if(df[i, 'quan'] > 0 ) {
            row_ct <- row_ct + 1
           # tally_q the buys from each row 
            tally_q <- tally_q + df[i, 'quan']
            cat("BUY: Row no: ", row_ct, "Cost is: ", df[i, "basis"], ", tally_q is:", tally_q, "\n")
            
            # cat("BUY ", df[i,'quan'], " Row number is: ", row_ct, " The tally_q is: ", tally_q, "\n")
            }

            #  When a Sell occurs, Call remote fct to adjust the tally_q == tally_q_q
            if(df[i, 'quan'] < 0 )  {
                quan_sold <- df[i, 'quan']
                row_ct <- row_ct + 1
               cat("SELL ", df[i,'quan'], " Row number is: ", row_ct, " The tally_q is: ", tally_q, " Sell Proceeds: ", df[i, 'basis'], "\n" )
               tally_q <- adj_tally_q(tally_q, quan_sold) 
                }
       }  # end For-loop 

        cat("\n")
        cat("The FINAL tally_q is :", tally_q, "\n")
        ct
} # end fct

#  These value will agree with the spreadsheet: BuySell.csv / .xlsx

In [34]:
make_buysell_csv()

The df has the following nrows:  41 
 
BUY: Row no:  1 Cost is:  4273.5 , tally_q is: 1050 
BUY: Row no:  2 Cost is:  5625 , tally_q is: 2050 
BUY: Row no:  3 Cost is:  5788 , tally_q is: 3050 
BUY: Row no:  4 Cost is:  10478.8 , tally_q is: 5050 
SELL  -1000  Row number is:  5  The tally_q is:  5050  Sell Proceeds:  -5512.4 
tally_q in the remote fct 5050 proceeds is:  -1000 
ADJUSTED tally_q - reduced by proceeds in remote fct 4050 
SELL  -1000  Row number is:  6  The tally_q is:  4050  Sell Proceeds:  -6682.55 
tally_q in the remote fct 4050 proceeds is:  -1000 
ADJUSTED tally_q - reduced by proceeds in remote fct 3050 
SELL  -1000  Row number is:  7  The tally_q is:  3050  Sell Proceeds:  -6255 
tally_q in the remote fct 3050 proceeds is:  -1000 
ADJUSTED tally_q - reduced by proceeds in remote fct 2050 
SELL  -1000  Row number is:  8  The tally_q is:  2050  Sell Proceeds:  -6190.1 
tally_q in the remote fct 2050 proceeds is:  -1000 
ADJUSTED tally_q - reduced by proceeds in remote

[1] 41

In [35]:
# stopper

In [36]:
    # remote fct to adjust the running stock tally_q (combine sum of Buys with the next Sell), return adjusted tally_q
    adj_tally_q <- function(tally_q, quan_sold) {
        cat("tally_q in the remote fct", tally_q, "The proceeds is: ", quan_sold, "\n")
        tally_q <- tally_q + quan_sold
        cat("ADJUSTED tally_q - reduced by proceeds in remote fct", tally_q, "\n")
       
        return(tally_q)        
        }       


In [37]:
# Identify the rows in the df where the proceedss occur

# Create counter vars
ct <- 0
tally_q <- 0
row_ct <<- 1
cat("The df has the following nrows: ", nrow(df), "\n \n")

# loop through the df
for(i in 1:nrow(df))  { 
    
     ct <- ct + 1
    # Sum the buy values from each row
    if(df[i, 'quan'] > 0 ) {
        
        # tally_q the buys from each row 
         tally_q <- tally_q + df[i, 'quan']
        cat("BUY. ", df[i,'quan'], " row number is: ", row_ct, " tally_q is: ", tally_q, "\n")
       
        row_ct <- row_ct + 1
        #print(paste0("BUY ", df[i,'quan'], " Row number is: ", row_ct, " The tally_q is: ", tally_q))       
        }
    
        
        #  When a Sell occurs, Call remote fct to adjust the tally_q == tally_q_q
        if(df[i, 'quan'] < 0 )  {
            quan_sold <- df[i, 'quan']
            row_ct <- row_ct + 1
           cat("SELL ", df[i,'quan'], " Row number is: ", row_ct, " The tally_q is: ", tally_q, " Sell quan_sold: ", df[i, 'basis'], "\n" )
           tally_q <- adj_tally_q(tally_q, quan_sold) 
            }
   }
 
cat("\n")
cat("The FINAL tally_q is :", tally_q, "\n")  # This value s/b 0 
ct

#  These value agree with the spreadsheet (tally_q_q on SELL line) See: BuySell.csv / .xlsx
#  These value agree with the spreadsheet: BuySell.csv / .xlsx

The df has the following nrows:  41 
 
BUY.  1050  row number is:  1  tally_q is:  1050 
BUY.  1000  row number is:  2  tally_q is:  2050 
BUY.  1000  row number is:  3  tally_q is:  3050 
BUY.  2000  row number is:  4  tally_q is:  5050 
SELL  -1000  Row number is:  6  The tally_q is:  5050  Sell quan_sold:  -5512.4 
tally_q in the remote fct 5050 The proceeds is:  -1000 
ADJUSTED tally_q - reduced by proceeds in remote fct 4050 
SELL  -1000  Row number is:  7  The tally_q is:  4050  Sell quan_sold:  -6682.55 
tally_q in the remote fct 4050 The proceeds is:  -1000 
ADJUSTED tally_q - reduced by proceeds in remote fct 3050 
SELL  -1000  Row number is:  8  The tally_q is:  3050  Sell quan_sold:  -6255 
tally_q in the remote fct 3050 The proceeds is:  -1000 
ADJUSTED tally_q - reduced by proceeds in remote fct 2050 
SELL  -1000  Row number is:  9  The tally_q is:  2050  Sell quan_sold:  -6190.1 
tally_q in the remote fct 2050 The proceeds is:  -1000 
ADJUSTED tally_q - reduced by proceed

[1] 41

## <font color = yellow> Convert the entire thing into a fct

In [38]:
 # adj_basis <- function(tally_q, proceeds) {
 #        print(paste0("In adj_basis fct. tally_q is: ", tally_q, " Sell is: ", proceeds))
 #        n <- 2
 #        tally_q <<- tally_q + n
 #        return(tally_q)
 #        }

In [39]:
# Function that reads BuySell.csv and will perform actions which agree with the values of (tally_q_q) on Sell dates

make_buysell_csv <- function() {    

    # remote fct to test passing value from fct to remote to this remote
     adj_basis <- function(tally_q, quan_sold) {
        print(paste0("In adj_basis fct. tally_q is: ", tally_q, " Sell is: ", quan_sold))
        n <- 2
        tally_q <<- tally_q + n
        return(tally_q)
        }   

    # remote fct to adjust the running stock tally_q (combine sum of Buys with the next Sell), return adjusted tally_q
    adj_tally_q <- function(tally_q, quan_sold) {
        # cat("tally_q in the remote fct", tally_q, "\n")
        tally_q <- tally_q + quan_sold
        # cat("ADJUSTED tally_q - reduced by proceeds in remote fct", tally_q, "\n")
        # Fct call
        # adjusted <- adj_basis(tally_q, proceeds)
        # print(paste0("adjusted from remote fct is: ", adjusted))
        return(tally_q)        
        }       

    # Main body     

    # Create counter vars
    ct <- 0
    tally_q <<- 0
    tally_sales <<- 0
    row_ct <<- 0
    cat("The df has the following nrows: ", nrow(df), "\n \n")

    # loop through the df
    for(i in 1:nrow(df))  { 

         ct <- ct + 1
        # Sum the buy values from each row
        if(df[i, 'quan'] > 0 ) {
            row_ct <- row_ct + 1
           # tally_q the buys from each row 
            tally_q <- tally_q + df[i, 'quan']
            #cat("BUY: Row no: ", row_ct, " Cost ", df[i, "basis"], " tally_q is: ", tally_q, "\n")
            
            # print(paste0("BUY ", df[i,'quan'], " Row number is: ", row_ct, " The tally_q is: ", tally_q))       
            }

            #  When a Sell occurs, Call remote fct to adjust the tally_q == tally_q_q
            if(df[i, 'quan'] < 0 )  {
                quan_sold <- df[i, 'quan']
                row_ct <- row_ct + 1
                tally_sales <<- df[i, 'basis'] + tally_sales
                #cat("tally_sales is: ", tally_sales, "\n")
                cat("SELL ", df[i,'quan'], " Row number is: ", row_ct, " Sale quan is :", quan_sold, " Sell Proceeds: ", df[i, 'basis'], "\n" )
                tally_q <- adj_tally_q(tally_q, quan_sold) 
                }       
        
       }  # end For-loop 

        cat("\n")
        cat("The FINAL tally_q is :", tally_q, "\n")
        cat("The FINAL tally_sales is :", tally_sales, "\n")
        ct
} # end fct

#  These value will agree with the spreadsheet: BuySell.csv / .xlsx

In [40]:
#  Call the fct
make_buysell_csv()
#  These value will agree with the spreadsheet: BuySell.csv / .xlsx

The df has the following nrows:  41 
 
SELL  -1000  Row number is:  5  Sale quan is : -1000  Sell Proceeds:  -5512.4 
SELL  -1000  Row number is:  6  Sale quan is : -1000  Sell Proceeds:  -6682.55 
SELL  -1000  Row number is:  7  Sale quan is : -1000  Sell Proceeds:  -6255 
SELL  -1000  Row number is:  8  Sale quan is : -1000  Sell Proceeds:  -6190.1 
SELL  -3000  Row number is:  16  Sale quan is : -3000  Sell Proceeds:  -17565 
SELL  -6000  Row number is:  17  Sale quan is : -6000  Sell Proceeds:  -39690 
SELL  -3000  Row number is:  18  Sale quan is : -3000  Sell Proceeds:  -23448 
SELL  -1850  Row number is:  19  Sale quan is : -1850  Sell Proceeds:  -14032.25 
SELL  -1000  Row number is:  20  Sale quan is : -1000  Sell Proceeds:  -7816 
SELL  -3000  Row number is:  23  Sale quan is : -3000  Sell Proceeds:  -19653.9 
SELL  -500  Row number is:  26  Sale quan is : -500  Sell Proceeds:  -2922.5 
SELL  -3000  Row number is:  27  Sale quan is : -3000  Sell Proceeds:  -17835 
SELL  -2000

[1] 41

In [41]:
# stopper

### <font color = lime> New: Adding new summing and track variables

In [42]:
# remote fct to adjust the running Buy tally_q with the next Sell quan
#  & return adjusted tally_q
    adj_tally_q <- function(tally_q, quan_sold) {
        tally_q <- tally_q + quan_sold
        cat("ADJUSTED tally_q - reduced by proceeds in remote fct", tally_q, "\n")
        return(tally_q)        
        }       

In [43]:
# Identify the rows in the df where the proceedss occur

# Create counter vars
ct <- 0
tally_q <- 0
row_ct <<- 1
accum_buy_basis <<- 0  # sums the cost of the Buys
accum_sell_basis <<- 0  # sums the proceeds from Sells
accum_sell_quan <<- 0    # count the number of Sells
accum_buy_quan <<- 0     # count the number of Buys

df_sell <- data.frame() # 'date' = as.integer(0), "quan" = as.integer(0), "basis" = as.numeric(0)) # empty df
df_buy <- data.frame() # empty df



cat("The df has the following nrows: ", nrow(df), "\n \n")

# loop through the df
for(i in 1:nrow(df))  { 
    
    ct <- ct + 1
    # Loop if Buy value larger than zero
    if(df[i, 'quan'] > 0 ) { 
        
       # tally the Buy from each row 
       tally_q <- tally_q + df[i, 'quan']
       cat("BUY. ", df[i,'quan'], " row number is: ", row_ct, " tally_q is: ", tally_q, "\n") 
        accum_buy_basis <- accum_buy_basis + df[i, 'basis']
        accum_buy_quan <- accum_buy_quan + df[i, 'quan']
       row_ct <- row_ct + 1
        df_temp_buy <- data.frame('date' = df[i, 'date'], 'quan' = df[i, 'quan'], 'basis' = df[i, 'basis'])
        df_buy <- rbind(df_buy, df_temp_buy)
       }
              
    #  If row is a Sell event, execute remote fct and reduce tally_q by Sell quan
    if(df[i, 'quan'] < 0 )  {
       quan_sold <- df[i, 'quan']
       row_ct <- row_ct + 1
       cat("SELL ", df[i,'quan'], " Row number is: ", row_ct, " The tally_q is: ", tally_q, " Sell Proceeds: ", df[i, 'basis'], "\n" )
        accum_sell_basis <- accum_sell_basis + df[i, 'basis']
        accum_sell_quan <- accum_sell_quan + df[i, 'quan']
        df_temp_sell <- data.frame('date' = df[i, 'date'], 'quan' = df[i, 'quan'], 'basis' = df[i, 'basis'])
        df_sell <- rbind(df_sell, df_temp_sell)
                                            
       # fct call
       tally_q <- adj_tally_q(tally_q, quan_sold) 
       }    
   }
 
cat("\n")
cat("The FINAL tally_q is :", tally_q, "\n")
ct


cat("\n", "The sum of the cost of the Buys is: ", accum_buy_basis, "\n")
cat("\n", "The sum of the proceeds of the Sells is: ", accum_sell_basis, "\n")

unadjusted_pl <- c(accum_buy_basis - abs(accum_sell_basis))
unadjusted_pl
cat("\n", "The total quantity of Buys is: ", accum_buy_quan, "\n")
cat("The total quantity of Sells is: ", accum_sell_quan, "\n")

#  These value agree with the spreadsheet (tally_q_q on SELL line) See: BuySell.csv / .xlsx

The df has the following nrows:  41 
 
BUY.  1050  row number is:  1  tally_q is:  1050 
BUY.  1000  row number is:  2  tally_q is:  2050 
BUY.  1000  row number is:  3  tally_q is:  3050 
BUY.  2000  row number is:  4  tally_q is:  5050 
SELL  -1000  Row number is:  6  The tally_q is:  5050  Sell Proceeds:  -5512.4 
ADJUSTED tally_q - reduced by proceeds in remote fct 4050 
SELL  -1000  Row number is:  7  The tally_q is:  4050  Sell Proceeds:  -6682.55 
ADJUSTED tally_q - reduced by proceeds in remote fct 3050 
SELL  -1000  Row number is:  8  The tally_q is:  3050  Sell Proceeds:  -6255 
ADJUSTED tally_q - reduced by proceeds in remote fct 2050 
SELL  -1000  Row number is:  9  The tally_q is:  2050  Sell Proceeds:  -6190.1 
ADJUSTED tally_q - reduced by proceeds in remote fct 1050 
BUY.  2000  row number is:  9  tally_q is:  3050 
BUY.  1000  row number is:  10  tally_q is:  4050 
BUY.  3000  row number is:  11  tally_q is:  7050 
BUY.  1500  row number is:  12  tally_q is:  8550 
BUY

[1] 41


 The sum of the cost of the Buys is:  190623.6 

 The sum of the proceeds of the Sells is:  -226104.5 


[1] -35480.85


 The total quantity of Buys is:  36950 
The total quantity of Sells is:  -36950 


In [44]:
head(df,3)

,date,type,co,quan,pps,basis,tally_q,tally_b,Sale
,<int>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<dbl>,<int>
1,44926,Owned,AMC,1050,4.070,4273.5,1050,4273.5,0
2,44946,Buy,AMC,1000,5.625,5625.0,2050,5625.0,0
3,44949,Buy,AMC,1000,5.788,5788.0,3050,5788.0,0


In [45]:
head(df_sell, 3)
head(df_buy,3)

,date,quan,basis
,<int>,<int>,<dbl>
1,44953,-1000,-5512.40
2,44960,-1000,-6682.55
3,44960,-1000,-6255.00


,date,quan,basis
,<int>,<int>,<dbl>
1,44926,1050,4273.5
2,44946,1000,5625.0
3,44949,1000,5788.0


In [46]:
cat("\n", "The total quantity of Buys is: ", accum_buy_quan, "\n")
cat("The total quantity of Sells is: ", accum_sell_quan, "\n")
nrow(df_sell)
nrow(df_buy)


 The total quantity of Buys is:  36950 
The total quantity of Sells is:  -36950 


[1] 18

[1] 23

In [47]:
# s/b 190,623.65
sum(df_buy$basis)

# s/b 226,104.50
sum(df_sell$basis)

sum(df$basis)

[1] 190623.6

[1] -226104.5

[1] -35480.85

### <font color = yellow> Subset refresher

In [48]:
ds <- subset(df, type == "Sell")
head(ds,3)

sum(ds$quan)
sum(ds$tally_b)
sum(ds$Sale)

,date,type,co,quan,pps,basis,tally_q,tally_b,Sale
,<int>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<dbl>,<int>
5,44953,Sell,AMC,-1000,5.5124,-5512.40,4050,-5512.40,-1000
6,44960,Sell,AMC,-1000,6.6826,-6682.55,3050,-6682.55,-1000
7,44960,Sell,AMC,-1000,6.2550,-6255.00,2050,-6255.00,-1000


[1] -36950

[1] -226104.5

[1] -36950

### <font color = yellow>  Better than subsetting:

In [115]:
#
buys <- df[df$quan > 0, ]  # Filter buys
head(buys, 3)

,date,type,co,quan,pps,basis,tally_q,tally_b,Sale
,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,44926,Owned,AMC,1050,4.070,4273.5,1050,4273.5,0
2,44946,Buy,AMC,1000,5.625,5625.0,2050,5625.0,0
3,44949,Buy,AMC,1000,5.788,5788.0,3050,5788.0,0


## <font color = yellow> All var values match the values from spreadsheets !
    
##### All's good.  Move on to dates and develop a fct for an adjusted cost basis

In [49]:
stopper

ERROR: Error in eval(expr, envir, enclos): object 'stopper' not found


## <font color = lime> Working with date values
    
##### Convert the df$date integer data type into a date data type 

##### <font color = yellow> The 1900 Date data type

In [54]:
# examine the integer data type 
n <- df_sell$date[1] - df_buy$date[1]
n
str(df_sell$date)
str(n)

[1] 27

 int [1:18] 44953 44960 44960 44960 44978 44979 44984 44984 44984 44988 ...
 int 27


In [55]:
# Convert the intger data type into the 1900 date format
x <- as.Date(df_sell$date[1], origin = "1899-12-30")
y <- as.Date(df_buy$date[1], origin = "1899-12-30")

x
y
z <- x - y
z

z > 31
z < 31
str(z)
attributes(z)

[1] "2023-01-27"

[1] "2022-12-31"

Time difference of 27 days

[1] FALSE

[1] TRUE

 'difftime' num 27
 - attr(*, "units")= chr "days"


$class
[1] "difftime"

$units
[1] "days"

# <font color = yellow> FIFO

In [103]:
# Identify the rows in the df where the proceedss occur

# Create counter vars
ct <- 0
tally_q <- 0
row_ct <<- 1
accum_buy_basis <<- 0  # sums the cost of the Buys
accum_sell_basis <<- 0  # sums the proceeds from Sells
accum_sell_quan <<- 0    # count the number of Sells
accum_buy_quan <<- 0     # count the number of Buys

df_sell <- data.frame() # 'date' = as.integer(0), "quan" = as.integer(0), "basis" = as.numeric(0)) # empty df
df_buy <- data.frame() # empty df

df_fifo <- df



cat("The df has the following nrows: ", nrow(df), "\n \n")

# loop through the df
for(i in 1:nrow(df))  { 
    
    ct <- ct + 1
    # Loop if Buy value larger than zero
    if(df[i, 'quan'] > 0 ) { 
        
       # tally the Buy from each row 
       tally_q <- tally_q + df[i, 'quan']
       # cat("BUY. ", df[i,'quan'], " row number is: ", row_ct, " tally_q is: ", tally_q, "\n") 
        # Cummulate var
        accum_buy_basis <- accum_buy_basis + df[i, 'basis']
        # cummulate var
        accum_buy_quan <- accum_buy_quan + df[i, 'quan']
        # counter 
        row_ct <- row_ct + 1
        # new temp df 
        df_temp_buy <- data.frame('date' = df[i, 'date'], 'quan' = df[i, 'quan'], 'basis' = df[i, 'basis'])
        #  Create new row of some variables for each of the Buy rows in df
        df_buy <- rbind(df_buy, df_temp_buy)
       }
              
    #  If row is a Sell event, execute remote fct and reduce tally_q by Sell quan
    if(df[i, 'quan'] < 0 )  {
        # new var for the quan from sold rows
       quan_sold <- df[i, 'quan']
       row_ct <- row_ct + 1
        cat("SELL ", df[i,'quan'], " Row number is: ", row_ct, " The tally_q is: ", tally_q, " Sell Proceeds: ", df[i, 'basis'], "\n" )
        # cummulative vars
        accum_sell_basis <- accum_sell_basis + df[i, 'basis']
        accum_sell_quan <- accum_sell_quan + df[i, 'quan']
        # temp df
        df_temp_sell <- data.frame('date' = df[i, 'date'], 'quan' = df[i, 'quan'], 'basis' = df[i, 'basis'])
        #  Create new row of some variables for each of the Sell rows in df
        df_sell <- rbind(df_sell, df_temp_sell)
                                            
       # fct call when quan[i] < 0 : fnt will reduce tally of quan by the neg sale quan 
       tally_q <- adj_tally_q(tally_q, quan_sold, i) 
       }    
   }
 
cat("\n")
cat("The FINAL tally_q is :", tally_q, "\n")
ct


cat("\n", "The sum of the cost of all Buys is: ", accum_buy_basis)
cat("\n", "The sum of the proceeds from all Sells is: ", accum_sell_basis, "\n")

unadjusted_pl <- c(accum_buy_basis - abs(accum_sell_basis))
cat("\n", "The unadjusted gain/<loss> from all transactions is: ", unadjusted_pl, "\n")

# cat("\n", "The total quantity of Buys is: ", accum_buy_quan, "\n")
# cat("The total quantity of Sells is: ", accum_sell_quan, "\n")

#  These value agree with the spreadsheet (tally_q_q on SELL line) See: BuySell.csv / .xlsx

The df has the following nrows:  41 
 
SELL  -1000  Row number is:  6  The tally_q is:  5050  Sell Proceeds:  -5512.4 
The quan_sold in the remote fct is:  -1000 
ADJUSTED tally_q - reduced by proceeds in remote fct 4050 
[1] "Quan Reduction Necessary"
SELL  -1000  Row number is:  7  The tally_q is:   Sell Proceeds:  -6682.55 
The quan_sold in the remote fct is:  -1000 
ADJUSTED tally_q - reduced by proceeds in remote fct  
[1] "Quan Reduction Necessary"
SELL  -1000  Row number is:  8  The tally_q is:   Sell Proceeds:  -6255 
The quan_sold in the remote fct is:  -1000 
ADJUSTED tally_q - reduced by proceeds in remote fct  
[1] "Quan Reduction Necessary"
SELL  -1000  Row number is:  9  The tally_q is:   Sell Proceeds:  -6190.1 
The quan_sold in the remote fct is:  -1000 
ADJUSTED tally_q - reduced by proceeds in remote fct  
[1] "Quan Reduction Necessary"
SELL  -3000  Row number is:  17  The tally_q is:    Sell Proceeds:  -17565 
The quan_sold in the remote fct is:  -3000 
ADJUSTED tall

[1] 41


 The sum of the cost of all Buys is:  190623.6
 The sum of the proceeds from all Sells is:  -226104.5 

 The unadjusted gain/<loss> from all transactions is:  -35480.85 


In [102]:
# remote fct called when a sale occurs. 
# fct will reduce tally of quan by the neg sale quan amount.  
#  Then, return adjusted tally_q

    adj_tally_q <- function(tally_q, quan_sold, i) {
        tally_q <- tally_q + quan_sold
        cat("The quan_sold in the remote fct is: ", quan_sold, "\n")
        cat("ADJUSTED tally_q - reduced by proceeds in remote fct", tally_q, "\n")
        
        if(df[i, 'quan'] > quan_sold) {
            print("Quan Reduction Necessary")
            test <- df[i, 'quan'] - quan_sold
            if(test > 0 ) {
              df[i, 'quan'] <- test
                }
            
            # Update the quantity in the current row
            # assign new value to quan
            
            return()
            }       
       
        return(tally_q)        
        }  

In [101]:
head(df_fifo, 3)

,date,type,co,quan,pps,basis,tally_q,tally_b,Sale
,<int>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<dbl>,<int>
1,44926,Owned,AMC,38000,4.070,4273.5,1050,4273.5,0
2,44946,Buy,AMC,1000,5.625,5625.0,2050,5625.0,0
3,44949,Buy,AMC,1000,5.788,5788.0,3050,5788.0,0


In [104]:
# processes the sales against the buys in a FIFO manner and stops if there are not enough buys to cover a sale. The result is a list containing the remaining buys and processed sells. Adjust the function as needed for your specific requirements. 


fifo_cost_basis <- function(df) {
  df <- df[order(as.Date(df$date, origin = "1899-12-30")), ]  # Sort by date
  buys <- df[df$quan > 0, ]  # Filter buys
  sells <- df[df$quan < 0, ]  # Filter sells
  
# for(i in 1:nrow(df))  { 
  for (sell_row in 1:nrow(sells)) {
    sell_qty <- abs(sells[sell_row, 'quan'])
    while (sell_qty > 0 && nrow(buys) > 0) {
      buy_qty <- buys[1, 'quan']
      if (buy_qty <= sell_qty) {
        # Entire buy quantity is sold
          
        sell_qty <- sell_qty - buy_qty
        buys <- buys[-1, ]  # Remove the used buy
      } else {
        # Partial buy quantity is sold
        buys[1, 'quan'] <- buy_qty - sell_qty
        sell_qty <- 0
      }
    }
    if (sell_qty > 0) {
      stop("Not enough buys to cover the sells")
    }
  }
  return(list(buys = buys, sells = sells))
}

# Usage
df <- read.csv("BuySell.csv", header = TRUE, sep = ",")
result <- fifo_cost_basis(df)


In [114]:
#str(result)
head(result$sells,5)

,date,type,co,quan,pps,basis,tally_q,tally_b,Sale
,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
5,44953,Sell,AMC,-1000,5.5124,-5512.40,4050,-5512.40,-1000
6,44960,Sell,AMC,-1000,6.6826,-6682.55,3050,-6682.55,-1000
7,44960,Sell,AMC,-1000,6.2550,-6255.00,2050,-6255.00,-1000
8,44960,Sell,AMC,-1000,6.1901,-6190.10,1050,-6190.10,-1000
16,44978,Sell,AMC,-3000,5.8550,-17565.00,11850,-17565.00,-3000


In [119]:
fifo <- function(buy_qty, sell_qty, sell_proceeds,buy_basis) {
    cat(buy_qty, sell_qty, sell_proceeds,buy_basis, "\n")
    return()
    }

In [120]:
# processes the sales against the buys in a FIFO manner and stops if there are not enough buys to cover a sale. The result is a list containing the remaining buys and processed sells. Adjust the function as needed for your specific requirements. 


fifo_cost_basis <- function(df) {
  df <- df[order(as.Date(df$date, origin = "1899-12-30")), ]  # Sort by date
  buys <- df[df$quan > 0, ]  # Filter buys
  sells <- df[df$quan < 0, ]  # Filter sells
  
# for(i in 1:nrow(df))  { 
  for (sell_row in 1:nrow(sells)) {
    sell_qty <- abs(sells[sell_row, 'quan'])
    buy_basis <- buys[1, 'basis']
    sell_proceeds <- abs(sells[sell_row, 'basis'])
    while (sell_qty > 0 && nrow(buys) > 0) {
      buy_qty <- buys[1, 'quan']
      if (buy_qty <= sell_qty) {
        # Entire buy quantity is sold         
        # fct call
          
          
          fifo(buy_qty, sell_qty, sell_proceeds,buy_basis)
          
        sell_qty <- sell_qty - buy_qty
        buys <- buys[-1, ]  # Remove the used buy
      } else {
        # Partial buy quantity is sold
        buys[1, 'quan'] <- buy_qty - sell_qty
        sell_qty <- 0
      }
    }
    if (sell_qty > 0) {
      stop("Not enough buys to cover the sells")
    }
  }
  return(list(buys = buys, sells = sells))
}

# Usage
df <- read.csv("BuySell.csv", header = TRUE, sep = ",")
result <- fifo_cost_basis(df)


50 1000 6682.55 4273.5 
50 1000 6255 5625 
50 1000 6190.1 5788 
1050 3000 17565 10478.8 
50 6000 39690 9490 
1000 5950 39690 9490 
3000 4950 39690 9490 
1500 1950 39690 9490 
300 450 39690 9490 
1850 3000 23448 9420 
1000 1000 7816 19120.4 
1500 3000 17835 32945.5 
1000 1500 17835 32945.5 
400 500 2860 17832.39 
100 100 2860 17832.39 
1000 2000 9830 4295 
1000 5000 24725 8614.8 
1000 4000 24725 8614.8 
1000 3000 24725 8614.8 
2000 2000 24725 8614.8 
400 1000 4582.8 2004 
100 1100 4994 3811.5 
1000 1000 4994 3811.5 


In [123]:
buys <- df[df$quan > 0, ]
buy_qty <- buys[1, 'quan']
buy_qty

[1] 1050